Most labeling tools out there do not allow user to create custom labels. Purpose of this project is to give you a framework which you can edit so you don't have to start from scratch like I did.
This is a sample framework forlabeling custom labels for small scale labeling. We used this framework to label two polynomials to an image. Polynomial labels are the coefficients of the polynomial. You can edit the code to make it label anything you want such as face landmark points etc.

In [1]:
import sys
import math
import cv2
import numpy as np
import pandas as pd
import pickle

These are the helper functions. *on_mouse* function is a callback function for mouse click. *get_poly* fitsh a second order polynomial from points given as input and returns its coefficients. *get_poly_points* calculates back the points from the polynomial coefficients. This fınction is used for visualization of polynomial. *draw_lines* uses *get_poly_points* to get polynomial points and visualizes it.

In [2]:
def on_mouse(event, x, y, flags, params):# Callback for mouse event. 
        if event == cv2.EVENT_LBUTTONDOWN:# If we left clicked
             #print('Mouse Position: '+str(x)+', '+str(y)) #For debug purposes
             global point # Define this as global so we can use it outside this function as well
             point = [x, y]#Store cursor position
             # Don't know how but we can access variables outside the function as well.
             cv2.circle(imag,(x,y),3,(255,0,0),-1)#Draw the point so we can see it.
             draw_lines(imag,poly_left,poly_right,w)#Redraw the polynomials. Just in case :)
             cv2.imshow('image', imag)# Show the image
def get_poly_points(poly,h,num_poly=10):
    #I don't think I need to explain this.
    y = np.arange(0, h, h/num_poly)
    c=poly[0]
    b=poly[1]
    a=poly[2]
    x = a*y*y+b*y+c
    return np.vstack([x, y]).T.reshape((-1,1,2))
def get_poly(pointsr):
    #This one is pretty self explainatory
    points =np.array(pointsr)
    #print(points[:,0])
    x1=points[:,0]
    y1=points[:,1]
    return np.polynomial.polynomial.polyfit(y1 ,x1, 2)
def draw_lines(imag,poly_left,poly_right,h):
    #Get points from polynomial coefficients. For our application we had left polynomial and right polynomial.
    poly_pts_left = get_poly_points(poly_left,h,100)
    poly_pts_right = get_poly_points(poly_right,h,100)
    #Draw obtained points.
    cv2.polylines(imag,[np.int32(poly_pts_left)],False,(0,0,255),3)
    cv2.polylines(imag,[np.int32(poly_pts_right)],False,(0,255,255),3)

A CSV file will be used to store our labels. Before you start labeling, you must create an initial CSV file that contains all your image paths and an initial value for labels. We will overwrite these initial values when we label. You can create this CSV file by using "create_csv.py" script. You have to edit your folder name accordingly in the script because I am too lazy to implement passing command line arguments. :)
If you labeled some images and decided to continue tomorrow. You must not recreate CSV file!! (I know it is obvious but just in case)

In [3]:
# Name of the CSV file to be edited. If this first time of labeling you should have created a empty and default 
#csv file containing all image paths and default label values with "Create_csv.py ".
csv_file="dataset.csv"
csv_frame=pd.read_csv(csv_file)#Lets read that CSV file with pandas
image_paths = csv_frame.iloc[:, 0] # First entry is our image paths
label_frame = csv_frame.iloc[:, 1:] # The rest is our labels
#Let's check if everything is OK by getting an arbitrary sample. 4 is my lucky number.
print(image_paths[4])
img = cv2.imread(image_paths[4])
cv2.imshow("img",img)
cv2.waitKey(0)
cv2.destroyAllWindows()

/home/user/Desktop/Huzeyfe_KILIC/custom_labelig_framework/frames1/frame0239.jpg


If everything is alright until this far. We can start labeling our images.

In [6]:
#Just initialize some parameters
points = []
#Pickle will be used for autosave
filename="autosave_csn.pckl"
outfile = open(filename,'wb')
#start with first image
i=0
img = cv2.imread(image_paths[i])
h,w,c=img.shape
#label = labels.iloc[i,:].to_numpy()
labels = label_frame.iloc[:,:].to_numpy().astype(np.float)#You need to convert from pandas to numpy. My data type is float.
imag=img.copy()#Just get a copy to do manupilations
cv2.namedWindow('image')#Create our labeling window
poly_left=labels[i,0:3]#My first 3 labels are coefficients of left polynomial
poly_right=labels[i,3:6]#My last 3 labels are coefficients of right polynomial
draw_lines(imag,poly_left,poly_right,w)#Lets draw the stored labels.
cv2.imshow('image', imag)
while True:
    imag=img.copy()
    cv2.setMouseCallback('image', on_mouse, 0)#Add callback function for mouse licks
    #Since we are in a loop we redraw on every iteration
    for j in range(len(points)):#Draw the points we stored so far
        cv2.circle(imag,(points[j][0],points[j][1]),3,(255,0,0),-1)
    draw_lines(imag,poly_left,poly_right,h)#Draw the polinomials we fit.
    key = cv2.waitKey(1) & 0xFF #Detect keyboard key press
    # By following the below convention, add your desired key press action.
    # Maybe pressing 1 assigns selected point to first landmark etc.
    if key == ord("e"):
        #Go to next image
        i=i+1
        if i>label_frame.shape[0]-1:#If this is our last image. Don't go further
            i=label_frame.shape[0]-1
        img= cv2.imread(image_paths[i])#Read our new image
        pickle.dump(labels,outfile)#Autosave
        #Get our stored labels
        poly_left=labels[i,0:3]
        poly_right=labels[i,3:6]
        #Create a copy for manupilations.
        imag=img.copy()
        #Drwa the stored polynomials
        draw_lines(imag,poly_left,poly_right,h)
        cv2.imshow('image', imag)
    if key == ord("q"):
        #Go to previous image
        i=i-1
        if i<0:#If this is our first image. Don't go back
            i=0
        #Read new image
        img = cv2.imread(image_paths[i])
        pickle.dump(labels,outfile)#autosave
        #Get stored labels and draw them
        poly_left=labels[i,0:3]
        poly_right=labels[i,3:6]
        imag=img.copy()
        draw_lines(imag,poly_left,poly_right,h)
        cv2.imshow('image', imag)
    if key == ord("s"):
        #This adds the point selected by my mouse click to list of points I will use to fit a polynomial.
        points.append(point)
        #print(points) #debug
    if key == ord("z"):
        #This resets labels and points for this image.
        #So if we make mistake. We can delete it.
        points=[]
        poly_left=[0,0,0]
        poly_right=[0,0,0]
        labels[i,0:3]=poly_left
        labels[i,3:6]=poly_right
        imag=img.copy()
    if key == ord("a"):
        #This fits left polynomil from the point list  I created earlier
        if len(points)<2:#If there are not enough points Do nothing
            continue
        imag=img.copy()#Get new image
        poly_left=get_poly(points)#Fit polynomial with the points provided
        #print(poly_left)#debug
        draw_lines(imag,poly_left,poly_right,h)#redraw the polynomials
        points=[]#Reset the point storage
        cv2.imshow('image', imag)
        labels[i,0:3]=poly_left#assign new label to storage
    if key == ord("d"):
        #This fits right polynomil from the point list  I created earlier
        if len(points)<2:#If there are not enough points Do nothing
            continue
        imag=img.copy()#Get new image
        poly_right=get_poly(points)#Fit polynomial with the points provided
        #print(poly_right) #Debug
        points=[]#Reset the point storage
        draw_lines(imag,poly_left,poly_right,h)#redraw the polynomials
        labels[i,3:6]=poly_right #assign new label to storage
        cv2.imshow('image', imag)
    # if the 'c' key is pressed, break from the loop
    if key == ord("c"):
        #This is exit
        label_frame.iloc[:,:]=labels#ssign Label storage to frame
        break
    elif key == ord("k"):
        #Exit without saving
        break        
csv_frame= pd.concat([image_paths, label_frame],axis = 1)#Recrate CSV frame
csv_frame.to_csv(csv_file,index = False)#We dont want indexvalues in our CSV file
cv2.destroyAllWindows()

In [5]:
cv2.destroyAllWindows()#If there are syntax errors. Close the imshow

Once you complete your labeling you can read it back in your training script like show above